# **Amazon Lookout for Equipment** - 익명화한 익스펜더 데이터셋에 대한 데모
*파트 3: 모델 훈련*

In [1]:
BUCKET = '<YOUR_BUCKET_NAME_HERE>'
PREFIX = 'data'

## 초기화
---
데이터 준비 노트북을 수행하면 저장소는 이제 다음과 같이 구성됩니다.
```
/lookout-equipment-demo
|
+-- data/
|   |
|   +-- labelled-data/
|   |   \-- labels.csv
|   |
|   \-- training-data/
|       \-- expander/
|           |-- subsystem-01
|           |   \-- subsystem-01.csv
|           |
|           |-- subsystem-02
|           |   \-- subsystem-02.csv
|           |
|           |-- ...
|           |
|           \-- subsystem-24
|               \-- subsystem-24.csv
|
+-- dataset/
|   |-- labels.csv
|   |-- tags_description.csv
|   |-- timeranges.txt
|   \-- timeseries.zip
|
+-- notebooks/
|   |-- 1_data_preparation.ipynb
|   |-- 2_dataset_creation.ipynb
|   |-- 3_model_training.ipynb              <<< 본 노트북 <<<
|   |-- 4_model_evaluation.ipynb
|   \-- 5_inference_scheduling.ipynb
|
+-- utils/
    |-- lookout_equipment_utils.py
    \-- lookoutequipment.json
```

### 임포트

In [2]:
%%sh
pip -q install --upgrade pip
pip -q install --upgrade awscli boto3 sagemaker
aws configure add-model --service-model file://../utils/lookoutequipment.json --service-name lookoutequipment

In [3]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
import boto3
import os
import pandas as pd
import sagemaker
import sys
import warnings

# Lookout for Equipment API 호출 관리를 위한 Helper 함수
sys.path.append('../utils')
import lookout_equipment_utils as lookout

### 파라미터

In [5]:
warnings.filterwarnings('ignore')

DATA       = os.path.join('..', 'data')
LABEL_DATA = os.path.join(DATA, 'labelled-data')
TRAIN_DATA = os.path.join(DATA, 'training-data', 'expander')

ROLE_ARN = sagemaker.get_execution_role()
REGION_NAME = boto3.session.Session().region_name

이전 분석을 바탕으로 다음의 기간 범위를 사용합니다.

* **훈련 셋:** 2015년 1월 1일 - 2015년 8월 31일: Lookout for Equipment는 최소 180 일의 훈련 데이터를 필요로 합니다. 3월도 라벨이 태깅된 이상치 기간의 일부이므로 모델링 동작을 변경해서는 안됩니다. 
* **시험 셋:** 2015년 9월 1일 - 2015년 11월 30일 *(시험 셋에는 모델을 평가하기 위한 정상과 비정상 데이터 모두가 포함되어야 함)*

In [6]:
# 기간 범위를 불러옵니다.
timeranges_fname = os.path.join(DATA, 'timeranges.txt')
with open(timeranges_fname, 'r') as f:
    timeranges = f.readlines()
    
training_start   = pd.to_datetime(timeranges[0][:-1])
training_end     = pd.to_datetime(timeranges[1][:-1])
evaluation_start = pd.to_datetime(timeranges[2][:-1])
evaluation_end   = pd.to_datetime(timeranges[3][:-1])

print(f'Training period: from {training_start} to {training_end}')
print(f'Evaluation period: from {evaluation_start} to {evaluation_end}')

dataset_fname = os.path.join(DATA, 'dataset_name.txt')
with open(dataset_fname, 'r') as f:
    DATASET_NAME = f.readline()
    
print('Dataset used:', DATASET_NAME)

Training period: from 2015-01-01 00:00:00 to 2015-08-31 23:59:00
Evaluation period: from 2015-09-01 00:00:00 to 2015-11-30 23:59:00
Dataset used: lookout-demo-training-dataset


## 모델 훈련
---

In [7]:
# 모델 파라미터를 준비합니다.
lookout_model = lookout.LookoutEquipmentModel(model_name='lookout-demo-model-v1',
                                              dataset_name=DATASET_NAME,
                                              region_name=REGION_NAME)

# 훈련 / 검증 분할 일자를 지정합니다.
lookout_model.set_time_periods(evaluation_start,
                               evaluation_end,
                               training_start,
                               training_end)

# 라벨 데이터 위치를 지정합니다.
lookout_model.set_label_data(bucket=BUCKET, 
                             prefix=f'{PREFIX}/labelled-data/',
                             access_role_arn=ROLE_ARN)

# 훈련에 앞서 서비스가 데이터를 재샘플링할 비율을 설정합니다. 
lookout_model.set_target_sampling_rate(sampling_rate='PT5M')

In [8]:
# 모델을 실제로 생성하고 훈련시킵니다.
_ = lookout_model.train()

콘솔을 캡처한 것처럼 현재 훈련이 진행 중입니다.
    
![Training in progress](../assets/model-training-in-progress.png)

다음 셀을 통해 모델 훈련 진행 상황을 파악하세요. **본 모델을 훈련시키는데 약 1시간이 소요됩니다.** 훈련 시간을 결정하는 주요 동인은 다음과 같습니다. 
* 라벨 데이터셋의 라벨 개수 (주어진 경우)
* 데이터 포인트 개수: 이 개수는 샘플링 비율, 시계열 개수와 시간 범위에 따라 달라집니다.

In [9]:
lookout_model.poll_model_training()

2021-04-02 21:57:09 | Model training: IN_PROGRESS
2021-04-02 21:58:10 | Model training: IN_PROGRESS
2021-04-02 21:59:10 | Model training: IN_PROGRESS
2021-04-02 22:00:10 | Model training: IN_PROGRESS
2021-04-02 22:01:11 | Model training: IN_PROGRESS
2021-04-02 22:02:12 | Model training: IN_PROGRESS
2021-04-02 22:03:12 | Model training: IN_PROGRESS
2021-04-02 22:04:12 | Model training: IN_PROGRESS
2021-04-02 22:05:13 | Model training: IN_PROGRESS
2021-04-02 22:06:14 | Model training: IN_PROGRESS
2021-04-02 22:07:14 | Model training: IN_PROGRESS
2021-04-02 22:08:14 | Model training: IN_PROGRESS
2021-04-02 22:09:14 | Model training: IN_PROGRESS
2021-04-02 22:10:15 | Model training: IN_PROGRESS
2021-04-02 22:11:16 | Model training: IN_PROGRESS
2021-04-02 22:12:16 | Model training: IN_PROGRESS
2021-04-02 22:13:16 | Model training: IN_PROGRESS
2021-04-02 22:14:16 | Model training: IN_PROGRESS
2021-04-02 22:15:17 | Model training: IN_PROGRESS
2021-04-02 22:16:18 | Model training: IN_PROGRESS


현재 모델 훈련 중이며 본 노트북 처음에 선택한 검증 기간에 대해 백테스트 결과를 시각화해볼 수 있습니다. 

![Training complete](../assets/model-training-complete.png)

## 결론
---
이 노트북에서는 노트북 시리즈 파트 2에서 생성한 데이터셋을 사용하여 Lookout for Equipment 모델을 훈련시켰습니다.

이제 다음 노트북 중 한 가지를 실행해볼 수 있습니다. 
* 본 모델에 대해 **검증 데이터를 추출**하고 모델 결과에 대한 추가 분석을 수행할 다음 노트북으로 이동합니다.
* 또는 **추론 스케줄링 노트북**에서 모델을 구동하여 신규 데이터를 제공하고 결과를 저장합니다.